In [210]:
import pandas as pd
import numpy as np
import re
import datetime
import itertools
import json

#reading data
df = pd.read_csv('/Users/codyreece/Desktop/Repos/air-scrape2/air-scrape/Data/Bentonville_Ar.csv')

#removing empty columns
df = df.drop(columns=['host_joined','house_rules', 'specialties_1'])

In [211]:
df['location'].value_counts()

Fayetteville, Arkansas, United States         86
Bella Vista, Arkansas, United States          78
Bentonville, Arkansas, United States          51
Rogers, Arkansas, United States               42
Springdale, Arkansas, United States           23
Centerton, Arkansas, United States            11
Lowell, Arkansas, United States                4
Farmington, Arkansas, United States            2
Township 2, Arkansas, United States            1
Little Flock, Arkansas, United States          1
Washington County, Arkansas, United States     1
Name: location, dtype: int64

In [212]:
#cleaning data
def wrangle(X):

    X['price'] = X['price'].replace(r'\$', '' ,regex=True)
    X["price"] = X["price"].str.replace(",","").astype("float")
    X['guests'] = X['guests'].str.replace(r'\D', '',regex=True).astype("float")
    X['rooms'] = X['rooms'].str.replace(r'\D', '',regex=True)
    X['beds'] = X['beds'].str.replace(r'\D', '',regex=True).astype("float")
    X["baths"] = X["baths"].str.split(" ", expand=True)[0]
    X['badge'] = X['badge'].map({'SUPERHOST':1, 'empty':0})
    X = X.rename({'badge': 'superhost'}, axis='columns')
    X['rating'] = X['rating'].replace(regex='empty', value=0)
    X['review_count'] = X['review_count'].replace(regex='empty', value=0)
    X['review_count'] = X['review_count'].replace(r'\D', '',regex=True)

    return X

wrangle(df)

,url,name,header,guests,rooms,beds,baths,facilities,superhost,rating,review_count,price,sp_url,location,amenities
0,/rooms/568774443413437016?adults=4&check_in=20...,Newly Updated ☆ 1 Mile to Coler➱Downtown,Entire residential home in Downtown Bentonville,8.0,3,4.0,2,Wifi · Kitchen · Free parking · Self check-in,1,0,0,126.0,https://www.airbnb.com/s/Bentonville--Arkansas...,"Bentonville, Arkansas, United States","{""amenities"": [""Hair dryer"", ""Cleaning product..."
1,/rooms/39864150?adults=4&check_in=2022-03-30&c...,"Pintell Trails: Spacious 4 Bed, 3 Bath w/ Garage",Entire residential home in Bentonville,8.0,4,4.0,3,Wifi · Kitchen · Free parking · Self check-in,0,0,0,121.0,https://www.airbnb.com/s/Bentonville--Arkansas...,"Bentonville, Arkansas, United States","{""amenities"": [""Bathtub"", ""Hair dryer"", ""Washe..."
2,/rooms/23161645?adults=4&check_in=2022-03-30&c...,Modern Townhouse at Momentary/8th St Market,Entire townhouse in Downtown Bentonville,8.0,3,3.0,2.5,Wifi · Kitchen · Free parking · Self check-in,1,4.99,101,194.0,https://www.airbnb.com/s/Bentonville--Arkansas...,"Bentonville, Arkansas, United States","{""amenities"": [""Bathtub"", ""Hair dryer"", ""Shamp..."
3,/rooms/584082363006663396?adults=4&check_in=20...,True North | Back 40 Trail | Sleeps 6,Entire residential home in Bella Vista,6.0,3,3.0,2,Wifi · Kitchen · Free parking · Self check-in,1,0,0,122.0,https://www.airbnb.com/s/Bentonville--Arkansas...,"Bella Vista, Arkansas, United States","{""amenities"": [""Washer"", ""TV"", ""Air conditioni..."
4,/rooms/31653766?adults=4&check_in=2022-03-30&c...,Chic Shack,Entire residential home in Bentonville,6.0,3,3.0,2,Wifi · Kitchen · Free parking · Self check-in,1,4.90,31,129.0,https://www.airbnb.com/s/Bentonville--Arkansas...,"Bentonville, Arkansas, United States","{""amenities"": [""Hair dryer"", ""Shampoo"", ""Hot w..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,/rooms/14967588?adults=4&check_in=2022-03-30&c...,"4 bedrm, 3 full baths close to Razorback Stadium",Entire residential home in Fayetteville,8.0,4,4.0,3,Wifi · Kitchen · Free parking · Self check-in,0,4.58,19,550.0,https://www.airbnb.com/s/Bentonville--Arkansas...,"Fayetteville, Arkansas, United States","{""amenities"": [""Bathtub"", ""Hair dryer"", ""Clean..."
296,/rooms/52620078?adults=4&check_in=2022-03-30&c...,3 Bedroom home with beautiful back deck-lake view,Entire residential home in Springdale,6.0,3,25.0,Wifi,Wifi · Kitchen · Free parking · Self check-in,0,0,0,385.0,https://www.airbnb.com/s/Bentonville--Arkansas...,"Springdale, Arkansas, United States","{""amenities"": [""Lake view"", ""Washer"", ""Iron"", ..."
297,/rooms/12127971?adults=4&check_in=2022-03-30&c...,Beaver Lake Bed & Breakfast,Entire guest suite in Springdale,4.0,2,2.0,1,Wifi · Free parking,1,5.0,73,166.0,https://www.airbnb.com/s/Bentonville--Arkansas...,"Springdale, Arkansas, United States","{""amenities"": [""Hair dryer"", ""Shampoo"", ""Hot w..."
298,/rooms/51329961?adults=4&check_in=2022-03-30&c...,Serene 1 bedroom 1 bath cabin at Beaver Lake!,Entire cabin in Springdale,4.0,1,1.0,1,Wifi · Kitchen · Free parking · Self check-in,1,4.91,11,175.0,https://www.airbnb.com/s/Bentonville--Arkansas...,"Springdale, Arkansas, United States","{""amenities"": [""Bathtub"", ""Hair dryer"", ""Clean..."


In [213]:
LIST_FACILITIES = [
    'Kitchen',
    'Wifi',
    'Free parking',
    'Self check-in'
]


In [214]:
def make_facilities(df, list_facilities=LIST_FACILITIES):
    data_facilities = df.loc[:, ['url', 'facilities']]

    for _top_facility in list_facilities:
        data_facilities[f"facility_{_top_facility}"] = data_facilities['facilities'].fillna('empty').str.split(' · ').apply(lambda x: 1 if _top_facility in x else 0)
    data_facilities.drop('facilities', axis=1, inplace=True)

    return data_facilities

In [215]:
make_facilities(df)

,url,facility_Kitchen,facility_Wifi,facility_Free parking,facility_Self check-in
0,/rooms/568774443413437016?adults=4&check_in=20...,1,1,1,1
1,/rooms/39864150?adults=4&check_in=2022-03-30&c...,1,1,1,1
2,/rooms/23161645?adults=4&check_in=2022-03-30&c...,1,1,1,1
3,/rooms/584082363006663396?adults=4&check_in=20...,1,1,1,1
4,/rooms/31653766?adults=4&check_in=2022-03-30&c...,1,1,1,1
...,...,...,...,...,...
295,/rooms/14967588?adults=4&check_in=2022-03-30&c...,1,1,1,1
296,/rooms/52620078?adults=4&check_in=2022-03-30&c...,1,1,1,1
297,/rooms/12127971?adults=4&check_in=2022-03-30&c...,0,1,1,0
298,/rooms/51329961?adults=4&check_in=2022-03-30&c...,1,1,1,1


In [219]:
amenities_list = list(df.amenities)
amenities_list_string = " ".join(amenities_list)
amenities_list_string = amenities_list_string.replace('{', '')
amenities_list_string = amenities_list_string.replace('}', ',')
amenities_list_string = amenities_list_string.replace('"', '')
amenities_set = [x.strip() for x in amenities_list_string.lower().split(',')]
amenities_set = set(amenities_set)
amenities_set

{'',
 '120\\ hdtv with amazon prime video',
 '2 refrigerators available and also a dorm fridge at the boat dock. large cooler is also available to take to the dock for the day refrigerator',
 '2-5 years old',
 '40\\ hdtv with netflix',
 '42\\ hdtv',
 '43\\ hdtv with roku',
 '47\\ hdtv with roku',
 '5-10 years old',
 '50\\ hdtv',
 '50\\ hdtv with chromecast',
 '50\\ tv with chromecast',
 '55\\ hdtv with amazon prime video',
 '55\\ hdtv with disney+',
 '55\\ hdtv with netflix',
 '55\\ hdtv with roku',
 '55\\ tv with apple tv',
 '60\\ hdtv with netflix',
 '65\\ hdtv with roku',
 '65\\ tv',
 '65\\ tv with netflix',
 '70\\ hdtv with hulu',
 '70\\ hdtv with standard cable',
 '72\\ hdtv',
 '75\\ tv with amazon prime video',
 'air conditioning',
 'all new appliances refrigerator',
 'all new appliances stove',
 'amazon prime video',
 'amenities: [bathtub',
 'amenities: [city skyline view',
 'amenities: [cleaning products',
 'amenities: [courtyard view',
 'amenities: [garden view',
 'amenities: 

In [220]:
# Check
print(f"The dataset contains {len(df)} Airbnb listings\n")
print(f'Amenities with TV: {len(df[df["amenities"].str.contains("TV")])}')
print(f'Amenities with workspace: {len(df[df["amenities"].str.contains("workspace")])}')
print(f'Amenities with Coffee: {len(df[df["amenities"].str.contains("Coffee")])}')
print(f'Amenities with Bikes: {len(df[df["amenities"].str.contains("Bikes")])}')
print(f'Amenities with Dishwasher: {len(df[df["amenities"].str.contains("Dishwasher")])}')
print(f'Amenities with Breakfast: {len(df[df["amenities"].str.contains("Breakfast")])}')
print(f'Amenities with Bidet: {len(df[df["amenities"].str.contains("Bidet")])}')
print(f'Amenities with Crib: {len(df[df["amenities"].str.contains("Crib")])}')
print(f'Amenities with Clothing: {len(df[df["amenities"].str.contains("Clothing")])}')
print(f'Amenities with Free parking: {len(df[df["amenities"].str.contains("Free parking")])}')
print(f'Amenities with driveway parking: {len(df[df["amenities"].str.contains("driveway parking")])}')
print(f'Amenities with hot tub: {len(df[df["amenities"].str.contains("hot tub")])}')
print(f'Amenities with tub: {len(df[df["amenities"].str.contains("tub")])}')
print(f'Amenities with Children toys: {len(df[df["amenities"].str.contains("Children")])}')
print(f'Amenities with Private backyard: {len(df[df["amenities"].str.contains("Private backyard")])}')
print(f'Amenities with Courtyard: {len(df[df["amenities"].str.contains("Courtyard")])}')
print(f'Amenities with gym: {len(df[df["amenities"].str.contains("gym")])}')
print(f'Amenities with Gym: {len(df[df["amenities"].str.contains("Gym")])}')
print(f'Amenities with Pool: {len(df[df["amenities"].str.contains("Pool")])}')
print(f'Amenities with Pool table: {len(df[df["amenities"].str.contains("Pool table")])}')
print(f'Amenities with wifi: {len(df[df["amenities"].str.contains("wifi")])}')
print(f'Amenities with Wifi: {len(df[df["amenities"].str.contains("Wifi")])}')
print(f'Amenities with the word amenities: {len(df[df["amenities"].str.contains("amenities")])}')

The dataset contains 300 Airbnb listings

Amenities with TV: 287
Amenities with workspace: 196
Amenities with Coffee: 257
Amenities with Bikes: 9
Amenities with Dishwasher: 240
Amenities with Breakfast: 12
Amenities with Bidet: 6
Amenities with Crib: 19
Amenities with Clothing: 108
Amenities with Free parking: 256
Amenities with driveway parking: 17
Amenities with hot tub: 7
Amenities with tub: 175
Amenities with Children toys: 44
Amenities with Private backyard: 97
Amenities with Courtyard: 3
Amenities with gym: 2
Amenities with Gym: 17
Amenities with Pool: 27
Amenities with Pool table: 7
Amenities with wifi: 11
Amenities with Wifi: 280
Amenities with the word amenities: 287
